In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec
import IPython.display as display
import time
from PIL import Image

## Uploading file manually

In [ ]:
data = np.load('cropped_03ded186-0f82-41ca-aad6-c071fded0b82_0.npy', allow_pickle=True)

In [ ]:
# See type and shape
print(type(data))
try:
    print(data.shape)
except:
    pass

In [ ]:
!git clone https://github.com/TaatiTeam/MotionAGFormer.git
%cd MotionAGFormer

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D

def show3Dpose(vals, ax):
    ax.view_init(elev=15., azim=70)

    lcolor = (0, 0, 1)  # Blue
    rcolor = (1, 0, 0)  # Red

    I = np.array([0, 0, 1, 4, 2, 5, 0, 7, 8, 8, 14, 15, 11, 12, 8, 9])
    J = np.array([1, 4, 2, 5, 3, 6, 7, 8, 14, 11, 15, 16, 12, 13, 9, 10])
    LR = np.array([0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0], dtype=bool)

    for i in range(len(I)):
        x, y, z = [np.array([vals[I[i], j], vals[J[i], j]]) for j in range(3)]
        ax.plot(x, y, z, lw=2, color=lcolor if LR[i] else rcolor)

    RADIUS = 0.72
    RADIUS_Z = 0.7
    xroot, yroot, zroot = vals[0, 0], vals[0, 1], vals[0, 2]
    ax.set_xlim3d([-RADIUS + xroot, RADIUS + xroot])
    ax.set_ylim3d([-RADIUS + yroot, RADIUS + yroot])
    ax.set_zlim3d([-RADIUS_Z + zroot, RADIUS_Z + zroot])
    ax.set_aspect('auto')

    white = (1.0, 1.0, 1.0, 0.0)
    ax.xaxis.set_pane_color(white)
    ax.yaxis.set_pane_color(white)
    ax.zaxis.set_pane_color(white)

    ax.tick_params('x', labelbottom=False)
    ax.tick_params('y', labelleft=False)
    ax.tick_params('z', labelleft=False)


def create_pose_video(data, output_video_path="pose_video.mp4", frame_folder="pose_frames", fps=20):
    """
    Create a 3D pose animation video from a numpy array.
    
    Args:
        data (np.ndarray): Shape (num_frames, num_joints, 3)
        output_video_path (str): Output path for the .mp4 video
        frame_folder (str): Temporary folder to save frames
        fps (int): Frames per second for output video
    """
    os.makedirs(frame_folder, exist_ok=True)

    for i, frame in enumerate(data):
        fig = plt.figure(figsize=(6, 6))
        gs = gridspec.GridSpec(1, 1)
        ax = plt.subplot(gs[0], projection='3d')
        show3Dpose(frame, ax)
        frame_path = os.path.join(frame_folder, f"{i:04d}.png")
        plt.savefig(frame_path, dpi=100, bbox_inches='tight')
        plt.close(fig)

    # Get video dimensions from first frame
    img = cv2.imread(os.path.join(frame_folder, "0000.png"))
    height, width, _ = img.shape
    video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for i in range(len(data)):
        img = cv2.imread(os.path.join(frame_folder, f"{i:04d}.png"))
        video.write(img)

    video.release()
    print(f"Video saved to {output_video_path}")


In [ ]:
create_pose_video(data, output_video_path="test_pose_video.mp4")

In [ ]:
import boto3
import tempfile
def list_and_play_mp4_from_s3(bucket_name, prefix, aws_access_key_id=None, aws_secret_access_key=None, region_name='us-east-2'):
    if aws_access_key_id and aws_secret_access_key:
        s3 = boto3.client(
            's3',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            region_name=region_name
        )
    else:
        s3 = boto3.client('s3')

    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    count = 0
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                key = obj['Key']
                if key.endswith('.npy'):
                    print(f"\n[INFO] Processing: {key}")

                    with tempfile.NamedTemporaryFile(delete=False, suffix=".npy") as temp_file:
                        temp_path = temp_file.name
                    count += 1
                    if count == 5:
                        break
                    try:
                        s3.download_file(bucket_name, key, temp_path)
                        data = np.load(temp_path)
                        print(data)
                        base_name = os.path.splitext(os.path.basename(key))[0]
                        output_video_path = f"{base_name}_pose_video.mp4"
                        create_pose_video(data, output_video_path=output_video_path)
                    except:
                        print("FAILED")


list_and_play_mp4_from_s3('shadow-trainer-dev', 'pose_numpy/','','')
